<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - FunkSVD MRR</h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [1]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, funksvd
from lenskit.algorithms.funksvd import FunkSVD
from lenskit import topn
import pandas as pd
import numpy as np

ratings = pd.read_csv('C:\\Users\\Alejo\\Tesis\\Demo\\ml-100k\\u.data', sep='\t',
                      names=['user', 'item', 'rating', 'timestamp'])


def eval(aname, algo, train, test):
    fittable = util.clone(algo) # Object cloning means to create an exact copy of the original object.
    fittable = Recommender.adapt(fittable) 
    fittable.fit(train) # Entrenamos el algoritmo con el training dataset
    users = test.user.unique() # Devuelve un array con los users unicos en el testing dataset
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [2]:
def model_trainer(features,lrate,reg):
    all_recs = []
    test_data = []
    
    features=int(features)
    funksvd = FunkSVD(features=features, iterations=100, lrate=lrate, reg=reg, damping=5, range=None, bias=True) # define algorithm
    
    for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
        test_data.append(test) # save testing data
        all_recs.append(eval('FunkSVD', funksvd, train, test))
    
    all_recs = pd.concat(all_recs, ignore_index=True)
    test_data = pd.concat(test_data, ignore_index=True)
    rla = topn.RecListAnalysis()
    rla.add_metric(topn.recip_rank)
    results = rla.compute(all_recs, test_data)
                          
    return results["recip_rank"].mean()

In [4]:
from hyperopt import fmin, tpe, hp, STATUS_OK

def objective(params):
    features = params['features']
    lrate = params['lrate']
    reg = params['reg']
    metric = model_trainer(features,lrate,reg)
    print([features,lrate,reg,metric],',')
    return {'loss': -metric, 'status': STATUS_OK }

space={'features': hp.uniform('features', 1, 100),
       'lrate': hp.uniform('lrate', 0.0005, 0.005),
       'reg': hp.uniform('reg',0.005,0.05)}

best = fmin(objective, space, algo=tpe.suggest,max_evals=100)

[6.357685503917439, 0.0019134457975585905, 0.013038533793426184, 0.22671340510467425]                                  
,                                                                                                                      
[68.18495579919212, 0.002084748934025221, 0.02309082712991984, 0.13118833355999673]                                    
,                                                                                                                      
[45.36115705329773, 0.0012455886168323397, 0.005079901252535269, 0.12909258510796504]                                  
,                                                                                                                      
[63.95842126051561, 0.0012925341208050694, 0.047707146740671995, 0.06914016248944768]                                  
,                                                                                                                      
[9.853494589167136, 0.001676017831919198

[45.11010768042916, 0.003314478088726167, 0.0063748053205235, 0.2414366279852842]                                      
,                                                                                                                      
[45.1498078288355, 0.003342439826017723, 0.008418832683730181, 0.2394107284578299]                                     
,                                                                                                                      
[55.21819515127863, 0.004598999644834109, 0.005218212784948108, 0.1867122320518618]                                    
,                                                                                                                      
[62.24916277194585, 0.0029456995172124864, 0.007888107113495989, 0.2191441998917337]                                   
,                                                                                                                      
[50.442626001995876, 0.00306130324697938

In [6]:
metric = pd.DataFrame(array)
metric.columns = ['features','learning rate','reg','MRR']
metric.head(5)

,features,learning rate,reg,MRR
0,6.357686,0.001913,0.013039,0.226713
1,68.184956,0.002085,0.023091,0.131188
2,45.361157,0.001246,0.005080,0.129093
3,63.958421,0.001293,0.047707,0.069140
4,9.853495,0.001676,0.025013,0.207803


In [7]:
metric.sort_values(by=['MRR'],ascending=False).head(10)

,features,learning rate,reg,MRR
27,37.448812,0.002525,0.006576,0.251102
67,45.168426,0.002877,0.006496,0.244356
68,45.110108,0.003314,0.006375,0.241437
69,45.149808,0.003342,0.008419,0.239411
66,9.691071,0.002774,0.006433,0.237541
22,37.667527,0.002664,0.014776,0.236179
42,6.554967,0.001802,0.011271,0.235812
37,52.764181,0.002798,0.010859,0.233337
96,36.785951,0.002868,0.014814,0.232324
98,29.848453,0.002305,0.011566,0.232157


In [8]:
metric.to_csv('metric_MRR.csv', index=False)

In [2]:
metric = pd.read_csv('C:\\Users\\Alejo\\Tesis\\Algoritmos\\FunkSVD\\results\\metric_MRR.csv')
metric

,features,learning rate,reg,MRR
0,6.357686,0.001913,0.013039,0.226713
1,68.184956,0.002085,0.023091,0.131188
2,45.361157,0.001246,0.005080,0.129093
3,63.958421,0.001293,0.047707,0.069140
4,9.853495,0.001676,0.025013,0.207803
...,...,...,...,...
95,15.134420,0.004760,0.018207,0.160882
96,36.785951,0.002868,0.014814,0.232324
97,58.311634,0.002992,0.045466,0.137500
98,29.848453,0.002305,0.011566,0.232157


In [3]:
%matplotlib qt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
colormap = cm.hot
x = metric['reg']
y = metric['learning rate']
z = metric['features'] 
c = metric['MRR']


fig = plt.figure()

ax = fig.add_subplot(111, projection='3d')
img = ax.scatter(x, y, z, c=c, cmap=plt.hsv())
ax.set_xlabel('reg')
ax.set_ylabel('learning rate')
ax.set_zlabel('features')
fig.colorbar(img)
plt.show()